# Battle of the Pizzas

### Introduction

Pizza has always been favorite dish in the United States and there's no doubt about it. However, thw question "which city makes the best pizza?" has always been on going debate. The problem we aim to solve is to analyze the most popular Pizza cities and their pizzerias offer the best pizzas. Our target audience are international pizza lovers.

### Data

We will use the FourSquare API to collect data about locations of Pizza stores in 5 major US cities which are: New York,NY, Chicago, IL, New Haven, CT, San Francisco, CA and Orlando, FL. These are one of the most popular pizza  cities.